In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-ba7_3ri9
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-ba7_3ri9
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 326b0a57a80c6d0b4bad25ca7adf8138419ef1cb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10741 sha256=1c43b610d84440f376c57bb0b3d20f87e2433fdb6f06eb98d5fd7c81c461a971
  Stored in directory: /tmp/pip-ephem-wheel-cache-o_nx2vus/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [2]:
!pip show nvcc4jupyter

Name: nvcc4jupyter
Version: 1.2.1
Summary: Jupyter notebook plugin to run CUDA C/C++ code
Home-page: 
Author: 
Author-email: Andrei Nechaev <lyfaradey@yahoo.com>, Cosmin Stefan Ciocan <ciocan.cosmin98@gmail.com>
License: MIT License
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: 


In [3]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmppz928fo9".


In [5]:
%%cuda

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <random>

#define BLOCK_SIZE 256

// Structure for CSR matrix
typedef struct {
    int rows_count;
    int cols_count;
    int non_zero_count;
    float* values;
    int* col_indices;
    int* row_ptr;
} csr_matrix;

// Function to generate a random CSR matrix
csr_matrix generate_random_csr_matrix(int rows, int cols, float density) {
    csr_matrix matrix;
    matrix.rows_count = rows;
    matrix.cols_count = cols;
    matrix.row_ptr = (int*)malloc((rows + 1) * sizeof(int));

    // Generate random values for the matrix
    std::random_device rd;
    std::mt19937 gen(rd());
    std::uniform_real_distribution<> dis(1.0, 1000.0); // Random value between 1 and 1000
    std::uniform_int_distribution<> col_dis(0, cols - 1); // Random column index between 0 and cols - 1

    int max_non_zeros = (int)(density * rows * cols);
    matrix.non_zero_count = max_non_zeros;
    matrix.values = (float*)malloc(max_non_zeros * sizeof(float));
    matrix.col_indices = (int*)malloc(max_non_zeros * sizeof(int));

    int current_index = 0;
    matrix.row_ptr[0] = 0;
    for (int i = 0; i < rows; ++i) {
        int non_zeros_in_row = (int)(density * cols); // Determine number of non-zeros in this row
        matrix.row_ptr[i + 1] = matrix.row_ptr[i] + non_zeros_in_row; // Update row_ptr
        for (int j = 0; j < non_zeros_in_row; ++j) {
            matrix.values[current_index] = dis(gen); // Random value
            matrix.col_indices[current_index] = col_dis(gen); // Random column index
            ++current_index;
        }
    }

    return matrix;
}

// CUDA kernel for SpMV computation with memory coalescing and shared memory optimization
__global__ void csr_spmv_kernel_shared(int rows_count, int* row_ptr, int* col_indices, float* values, float* x, float* y) {
    // Define shared memory for values and column indices
    __shared__ float shared_values[BLOCK_SIZE];
    __shared__ int shared_col_indices[BLOCK_SIZE];

    // Calculate the thread index within the block
    int tid = threadIdx.x;
    int row = blockIdx.x * blockDim.x + tid;

    // Initialize the result for this thread's row
    float dot = 0.0f;

    // Loop over the tiles of the matrix
    for (int tile_index = 0; tile_index < (rows_count + BLOCK_SIZE - 1) / BLOCK_SIZE; ++tile_index) {
        // Load values and column indices for this tile into shared memory
        int col_index = tile_index * BLOCK_SIZE + tid;
        if (col_index < rows_count) {
            int offset = row_ptr[col_index];
            shared_values[tid] = values[offset + tid];
            shared_col_indices[tid] = col_indices[offset + tid];
        } else {
            shared_values[tid] = 0.0f;
            shared_col_indices[tid] = -1; // Invalid index
        }

        // Synchronize to make sure all threads have loaded the tile
        __syncthreads();

        // Perform the dot product within the tile
        for (int i = 0; i < BLOCK_SIZE; ++i) {
            if (shared_col_indices[i] != -1) { // Check for valid column index
                dot += shared_values[i] * x[shared_col_indices[i]];
            }
        }

        // Synchronize before loading the next tile
        __syncthreads();
    }

    // Write the result back to global memory
    if (row < rows_count) {
        y[row] = dot;
    }
}

// Function to perform SpMV on GPU and measure performance
void gpu_csr_spmv_perf(const csr_matrix* matrix, const float* x, float* y) {
    // Allocate memory on GPU
    float *d_values, *d_x, *d_y;
    int *d_col_indices, *d_row_ptr;
    cudaMalloc((void**)&d_values, matrix->non_zero_count * sizeof(float));
    cudaMalloc((void**)&d_col_indices, matrix->non_zero_count * sizeof(int));
    cudaMalloc((void**)&d_row_ptr, (matrix->rows_count + 1) * sizeof(int));
    cudaMalloc((void**)&d_x, matrix->cols_count * sizeof(float));
    cudaMalloc((void**)&d_y, matrix->rows_count * sizeof(float));

    // Check for memory allocation errors
    if (!d_values || !d_col_indices || !d_row_ptr || !d_x || !d_y) {
        fprintf(stderr, "CUDA memory allocation error\n");
        exit(EXIT_FAILURE);
    }

    // Copy data to GPU
    cudaMemcpy(d_values, matrix->values, matrix->non_zero_count * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_col_indices, matrix->col_indices, matrix->non_zero_count * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_row_ptr, matrix->row_ptr, (matrix->rows_count + 1) * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_x, x, matrix->cols_count * sizeof(float), cudaMemcpyHostToDevice);

    // Launch kernel
    int threads_per_block = BLOCK_SIZE;
    int blocks_per_grid = (matrix->rows_count + threads_per_block - 1) / threads_per_block;

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventRecord(start);

    csr_spmv_kernel_shared<<<blocks_per_grid, threads_per_block>>>(matrix->rows_count, d_row_ptr, d_col_indices, d_values, d_x, d_y);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float milliseconds = 0;
    cudaEventElapsedTime(&milliseconds, start, stop);

    // Copy result back to CPU
    cudaMemcpy(y, d_y, matrix->rows_count * sizeof(float), cudaMemcpyDeviceToHost);

    // Check for kernel launch errors
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        fprintf(stderr, "CUDA kernel launch error: %s\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    // Free GPU memory
    cudaFree(d_values);
    cudaFree(d_col_indices);
    cudaFree(d_row_ptr);
    cudaFree(d_x);
    cudaFree(d_y);

    // Calculate total number of floating-point operations
    double total_operations = (double)matrix->non_zero_count * 2; // Assuming one multiplication and one addition per non-zero element

    // Calculate GFLOPS
    double seconds = milliseconds / 1000.0; // Convert milliseconds to seconds
    double gflops = (total_operations / seconds) / 1e9; // Divide by elapsed time in seconds and 1 billion
    double throughput = total_operations / seconds; // Throughput in operations per second

    // Output performance metrics
    printf("Elapsed Time: %f milliseconds\n", milliseconds);
    printf("GFLOPS: %f\n", gflops);
    printf("Throughput: %f OPS\n", throughput);
}

int main() {
    // Define matrix dimensions and density
    int size = 1 << 15; // 2^15
    float density = 0.1; // Density of non-zero elements in the matrix (10%)

    // Generate a random CSR matrix
    csr_matrix matrix = generate_random_csr_matrix(size, size, density);

    // Allocate memory for input and output vectors
    float* x = (float*)malloc(size * sizeof(float));
    float* y = (float*)malloc(size * sizeof(float));

    // Check for memory allocation errors
    if (!x || !y) {
        fprintf(stderr, "Memory allocation error\n");
        exit(EXIT_FAILURE);
    }

    // Initialize input vector x (for demonstration, fill it with 1.0)
    for (int i = 0; i < size; ++i) {
        x[i] = 1.0f;
    }

    // Perform SpMV on GPU and measure performance
    printf("Performance metrics for random CSR matrix:\n");
    gpu_csr_spmv_perf(&matrix, x, y);

    // Free memory
    free(matrix.values);
    free(matrix.col_indices);
    free(matrix.row_ptr);
    free(x);
    free(y);

    return 0;
}

Performance metrics for random CSR matrix:
Elapsed Time: 10.840416 milliseconds
GFLOPS: 19.809975
Throughput: 19809974903.137856 OPS

